<a href="https://colab.research.google.com/github/wang-zijie/Data-Integration-Deep-Learning/blob/main/Data_Integration_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.3.1
!pip install pytorch-lightning==0.9.0

     |████████████████████████████████| 1.1MB 15.8MB/s 
     |████████████████████████████████| 1.2MB 53.4MB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
     |████████████████████████████████| 3.0MB 30.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3030d621f21f0c323aa426f180c8c863ce17189d655982ca676f5f9409849c90
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 409kB 14.3MB/s 
     |████████████████████████████████| 276kB 29.3MB/s 
     |████████████████████████████████| 829kB 30.4MB/s 
     |████████████████████████████████| 2.8MB 49.3MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=840d3282ff69e012acbef1973eb185e360bd6f74f91922d22f8832bfd71c2ab0
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b

In [ ]:
# !cat /proc/cpuinfo

In [ ]:
 !nvidia-smi

Wed Jan 13 21:45:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas


MAX_EPOCHS=3


In [ ]:
import json
with open('/content/drive/MyDrive/Research/dataset/imdb/imdb_data_cell_level_5000.json','r',encoding='utf-8') as f:
  train_data_json = [json.loads(i) for i in f.readlines()]
with open('/content/drive/MyDrive/Research/dataset/imdb/test_imdb_data_cell_level_5000.json','r',encoding='utf-8') as f:
  test_data_json = [json.loads(i) for i in f.readlines()]

In [ ]:
dats = []
for i in train_data_json:
  dats.extend(i['label_index'])
NUM_LABELS = len(set(dats))

In [ ]:
print(f'USING {NUM_LABELS} FOR the Classification')

USING 11475 FOR the Classification


In [ ]:
import re
import torch 
class DataIntegrationFormatter:
    @staticmethod
    def clean_ascii(text):
        # function to remove non-ASCII chars from data
        return ''.join(i for i in text if ord(i) < 128)
    
    def format(self):
        raise NotImplementedError
    
    def get_all_tokens(self):
        return [

        ]
    
    def _quote_token(self,token):
        return "<"+token+">"
    
    def _unquote_token(self,token):
        return "<"+token+"/>"

    def __call__(self,obj):
      content = []
      for k in obj:
        content.extend(
            [self._quote_token(k),obj[k],self._unquote_token(k)]
        )
        return ''.join(content)
        
    def decompile(self,formatted_text):
        raise NotImplementedError
        
formatter = DataIntegrationFormatter()

In [ ]:
from torch.utils.data import (
    random_split,
    DataLoader,
    RandomSampler,
    Subset,
    TensorDataset
)
import torch
from transformers import (
    BertTokenizer
)
from pytorch_lightning import Trainer, seed_everything
seed_everything(42)


class DocumentDataPreprocessor():
    """DocumentDataPreprocessor 
    This will manage the tokenization of documents that are formatted Using the 
    ResearchPaperStyleFormater. It will also prepare the XY dataset with attention mask
    
    This ensures that Tokens are set correctly set and preprocessinig can happen in different styles. 
    """
    CLASS_TOKEN = '[CLS]'
    SEP_TOKEN = '[EOS]'
    SPECIAL_TOKENS = []

    def __init__(self,tokenizer:BertTokenizer,\
                formatter=DataIntegrationFormatter(),\
                column_split_order=[],
                ):

        self.tokenizer = tokenizer
        special_tokens_dict = {'cls_token': self.CLASS_TOKEN,'pad_token':self.SEP_TOKEN}
        self.tokenizer.add_special_tokens(special_tokens_dict)

        self.formatter = formatter
        tokens = self.formatter.get_all_tokens()
        # self.tokenizer.add_tokens(tokens)
        # # File to save column Order so that inference of the same model can be done easily later
        # self.column_split_order = []
        # self.column_split_save_file_name = 'column_order.json'

    def __call__(self,content_text):
        return self.get_tokenized_text(content_text)

    def save_vocabulary(self,out_dir):
        self.tokenizer.save_vocabulary(out_dir)

    
    def prepare_text_tensors(self,documents,max_length=40):
        documents = documents
        attention_mask = []
        input_ids = []
        # From https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow
        # For every Document...
        # No need for explicit attention mask extraction in GPT2
        # https://github.com/huggingface/transformers/issues/808#issuecomment-522932583
        for document in documents:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            encoded_dict = self.tokenizer.encode_plus(
                                document,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = max_length,           # Pad & truncate all sentences.
                                padding = 'max_length',
                                truncation=True,
                                return_attention_mask=True,
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )
            
            # "Add the encoded sentence to the list". 
            input_ids.append(encoded_dict['input_ids'])
            attention_mask.append(encoded_dict['attention_mask'])
        input_ids = torch.cat(input_ids,dim=0)
        attention_mask = torch.cat(attention_mask,dim=0)
        # And its attention mask (simply differentiates padding from non-padding).
        return input_ids,attention_mask

    def prepare_classification_dataset(self,documents,labels,max_length=1024)->TensorDataset:
        """prepare_dataset 
        The function will create the dataset into text_tensors,attention_mask,label
        It expects Formatted Documents.
        """
        documents = documents
        # One-hot label conversion
        column_split_order = labels.str.get_dummies().columns.tolist()
        labels = labels.str.get_dummies().values.tolist()
        
        attention_mask = []
        input_ids = []
        # From https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow
        # For every Document...
        # No need for explicit attention mask extraction in GPT2
        # https://github.com/huggingface/transformers/issues/808#issuecomment-522932583
        for document in documents:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            encoded_dict = self.tokenizer.encode_plus(
                                document,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = max_length,           # Pad & truncate all sentences.
                                padding = 'max_length',
                                truncation=True,
                                return_attention_mask=True,
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )
            
            # "Add the encoded sentence to the list". 
            input_ids.append(encoded_dict['input_ids'])
            attention_mask.append(encoded_dict['attention_mask'])
            # And its attention mask (simply differentiates padding from non-padding).

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids,dim=0)
        attention_mask = torch.cat(attention_mask,dim=0)
        labels = torch.tensor(labels)
        return TensorDataset(input_ids, attention_mask,labels),column_split_order
    
    def get_label_tensors(self,labels):
        column_split_order = labels.str.get_dummies().columns.tolist()
        labels = labels.str.get_dummies().values.tolist()
        labels = torch.tensor(labels)
        return labels,column_split_order


    def get_tokenized_text(self,content_text,max_length=1024,pad_to_max_length=True):
        attention_mask = []
        input_ids = []
        encoded_dict = self.tokenizer.encode_plus(
                    content_text,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = max_length,           # Pad & truncate all sentences.
                    padding = 'max_length',
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors = 'pt',     # Return pytorch tensors.
            )
    
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        attention_mask.append(encoded_dict['attention_mask'])
        # And its attention mask (simply differentiates padding from non-padding).
        input_ids = torch.cat(input_ids,dim=0)
        attention_mask = torch.cat(attention_mask,dim=0)
        return input_ids,attention_mask
    

    @staticmethod
    def split_dataset(dataset,train_percent=0.9):
        # Create a split in train-validation 
        # Calculate the number of samples to include in each set.
        if train_percent > 1:
            raise Exception('Training Percentage cannot be > 1')
        train_size = int(train_percent * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        return train_dataset,val_dataset

In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from transformers import BertLMHeadModel, BertForSequenceClassification,BertConfig,BertModel
# from transformers import GPT2LMHeadModel,GPT2PreTrainedModel,GPT2Model
from transformers import AdamW,get_linear_schedule_with_warmup,get_cosine_with_hard_restarts_schedule_with_warmup

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

In [ ]:
# country_region,US,state_subregion,North Carolina,Hoke County,date,,confirmed,,long,lat
# <country_region>US</country_region><state_subregion>North Carolina,Hoke County</state_subregion><date></date>

In [ ]:
class GPT2NormalisationHead(nn.Module):
  def __init__(self,num_labels=NUM_LABELS,ff_layers=3840,ff_hidden_dims=512):
    super(GPT2NormalisationHead,self).__init__()
    self.CNN_Max = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
    self.CNN_Avg = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=2, stride=2),
    )
    self.ff_head = nn.Sequential(
        nn.Linear(ff_layers,ff_hidden_dims),
        nn.Linear(ff_hidden_dims,num_labels)
    )
    self.final_sigmoid = nn.Sigmoid()
    self.flatten = Flatten()

  def forward(self,hidden):
    

    hidden = hidden.unsqueeze(1)
    # Convolve the hidden vectors
    max_op = self.CNN_Max(hidden)
    avg_op = self.CNN_Avg(hidden)
    
    # Shift dimensions to basically do a matrix Transpose of Max/avg pooliing from CNN
    max_op = max_op.view(max_op.size(0),max_op.size(2),-1)
    avg_op = avg_op.view(avg_op.size(0),avg_op.size(2),-1)
    # Multiple the matrixes. 
    result_op = torch.mul(avg_op,max_op)
    # average of final result.
    result_op = self.flatten(result_op)
    # final softmax for result logits.
    # print(f"Shape Of Results {result_op.shape} {self.ff_head}")
    result_logits = self.ff_head(result_op) #self.final_sigmoid()
    return result_logits

class BERTClassifier(BertForSequenceClassification):
  def __init__(self, config,op_labels=3,ff_layers=5):
      config.output_attentions=True
      super(BERTClassifier, self).__init__(config)
      self.transformer = BertModel(config)
      print(f"Got ff_layers {ff_layers} {op_labels}")
      self.classification_head = GPT2NormalisationHead(num_labels=op_labels,ff_layers=ff_layers)
      self.init_weights()
  def forward(self,input_ids, position_ids=None, token_type_ids=None, lm_labels=None, attention_mask=None):
      transformer_op = self.transformer(input_ids, position_ids=position_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
      hidden,past,attentions = transformer_op
      class_logits = self.classification_head(hidden)
      return class_logits

class DataIntegrationClassificationMultiHeadModel(nn.Module):
    # https://huggingface.co/transformers/model_doc/gpt2.html#gpt2mdel
    # ff_layer depenedent on size of input sequence. 
    def __init__(self, pretrain_path,op_labels = NUM_LABELS,ff_layers=1280):
        super(DataIntegrationClassificationMultiHeadModel, self).__init__()
        self.classification_head = BERTClassifier.from_pretrained(pretrain_path,op_labels=op_labels,ff_layers=ff_layers) 

    def forward(self,input_ids, position_ids=None, token_type_ids=None, lm_labels=None, attention_mask=None):
        classification_head_logits = self.classification_head(input_ids, position_ids=position_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)
        return classification_head_logits

    def resize_tokens(self,tokenizer_size):
        self.classification_head.transformer.resize_token_embeddings(tokenizer_size)

In [ ]:
import heapq
def get_onehot_label_topk(scores, top_num,threshold = 0.0):
        '''

        get the top k score from testing result,
        use threshold to filter the irrelevant data.

        :param scores:  predicted scores for each classification class
        :param top_num: number of labels for each data(corrosponding to top k scores)
        :param threshold: score of irrelevant data < threshold

        :return:
            predicted_onehot_labels: Predict labels (onehot format)
        '''
        predicted_onehot_labels = []
        for score, topk in zip(scores, top_num):
            
            score = np.ndarray.tolist(score)
            # for score in scores:
            count = 0
            onehot_labels_list = [0] * len(score)
            max_num_index_list = list(map(score.index, heapq.nlargest(int(topk), score)))
            
            for i in max_num_index_list:
                onehot_labels_list[i] = 1
            predicted_onehot_labels.append(onehot_labels_list)

        return predicted_onehot_labels

In [ ]:

from pytorch_lightning import metrics
from pytorch_lightning.metrics.functional.classification import accuracy
from sklearn.metrics import average_precision_score,accuracy_score
class GPT2DataIntegrationClassificationModel(pl.LightningModule):
    def __init__(self,pretrain_path,tokenizer,loss_agg='mult',loss_sign=1,exponent_loss=False,powx_loss=False,pow=2):
      # Positive sign doesn't work for optimisation. It actually makes worse predictions than when it started at. 
        super(GPT2DataIntegrationClassificationModel,self).__init__()
        self.classification_transformer = DataIntegrationClassificationMultiHeadModel(pretrain_path)
        self.loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
        self.tokenizer = tokenizer
        self.classification_transformer.resize_tokens(len(tokenizer))
        self.loss_agg = loss_agg
        self.loss_sign = loss_sign
        self.exponent_loss = exponent_loss
        self.powx_loss=powx_loss
        self.pow=pow
        
    def forward(self,input_ids, position_ids=None, token_type_ids=None, lm_labels=None, attention_mask=None, labels=None):
        return self.classification_transformer(input_ids,position_ids=position_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)


    def training_step(self,batch,batch_nb):
        input_ids,attention_mask,multi_label_tensor,num_labels = batch
        multi_label_pred_logits = self(input_ids,attention_mask=attention_mask,labels=input_ids)
        if self.exponent_loss:
          loss = torch.exp(self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        elif self.powx_loss:
          loss = torch.pow(self.pow,self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        else:
          loss = self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign
        
        acc = self.get_accuracy(multi_label_pred_logits,multi_label_tensor,num_labels) # TODO :FIX ACCURACY CALCULATIONS
        acc1 = acc[0]
        # precision = acc[1]
        
        result = pl.TrainResult(loss,checkpoint_on=loss)
        result.log('train_loss', loss,logger=True,on_epoch=True,on_step=True,prog_bar=True)
        
        result.log('train_acc',torch.Tensor([acc1]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        # result.log('train_precision',torch.Tensor([precision]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        
        # print('train_acc',acc1)
        return result
        # return {'loss':loss,'train_loss': loss,'log':{'train_loss':loss}} 
    
    def validation_step(self,batch,batch_nb):
        input_ids,attention_mask,multi_label_tensor,num_labels = batch
        multi_label_pred_logits = self(input_ids,attention_mask=attention_mask,labels=input_ids)
        if self.exponent_loss:
          loss = torch.exp(self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        elif self.powx_loss:
          loss = torch.pow(self.pow,self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        else:
          loss = self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign        
        acc = self.get_accuracy(multi_label_pred_logits,multi_label_tensor,num_labels) # TODO :FIX ACCURACY CALCULATIONS
        acc1 = acc[0]
        # precision = acc[1]
        
        result = pl.EvalResult(loss,checkpoint_on=loss)
        result.log('val_loss', loss,logger=True,on_epoch=True,on_step=True,prog_bar=True)
        
        result.log('val_acc',torch.Tensor([acc1]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        # result.log('val_precsion',torch.Tensor([precision]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        # print('val_acc',acc1)

        return result


    def test_step(self,batch,batch_nb):
        input_ids,attention_mask,multi_label_tensor,num_labels = batch
        multi_label_pred_logits = self(input_ids,attention_mask=attention_mask,labels=input_ids)
        if self.exponent_loss:
          loss = torch.exp(self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        elif self.powx_loss:
          loss = torch.pow(self.pow,self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign)
        else:
          loss = self.loss_fn(multi_label_pred_logits,multi_label_tensor)* self.loss_sign

        acc = self.get_accuracy(multi_label_pred_logits,multi_label_tensor,num_labels) # TODO :FIX ACCURACY CALCULATIONS
        
        acc1 = acc[0]
        # precison = acc[1]
        
        result = pl.EvalResult(loss,checkpoint_on=loss)
        result.log('test_loss', loss,logger=True,on_epoch=True,on_step=True,prog_bar=True)
        
        result.log('test_acc',torch.Tensor([acc1]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        # result.log('test_precision',torch.Tensor([precison]),logger=True,on_step=True,on_epoch=True,prog_bar=True)
        # print('test_acc',acc1)
        return result

    def configure_optimizers(self):
        optimizer = AdamW(self.classification_transformer.parameters(),
            lr = 0.0001, 
            eps = 1e-8 
        )
        num_minibatch_steps = 1938
        max_epochs = MAX_EPOCHS
        warmup = 0.2*max_epochs * num_minibatch_steps
        t_total = max_epochs * num_minibatch_steps
        num_cycles = 10
        lr_scheduler = get_linear_schedule_with_warmup(optimizer, warmup, t_total)
        return [optimizer],[lr_scheduler]

    
    @staticmethod
    def get_accuracy(res_op, target_multi_hot,num_labels_tensor):
        # https://stackoverflow.com/questions/61524717/pytorch-how-to-find-accuracy-for-multi-label-classification
        "" 
        with torch.no_grad():
            sigmoid_op = torch.sigmoid(res_op)
            # print('logits_tensor',sigmoid_op)
            sigmoid_op = sigmoid_op.cpu().numpy()
            # print('logits_numpy',sigmoid_op)
            # print('top_number',num_labels_tensor)
            # print(len(num_labels_tensor))
            num_labels_tensor = num_labels_tensor.cpu().numpy()
            target_multi_hot_numpy = target_multi_hot.cpu().numpy()
            # print('true_lable',target_multi_hot)
            predicted_label = get_onehot_label_topk(sigmoid_op,num_labels_tensor)
            
            # acc = accuracy(predicted_label,target_multi_hot)
            # acc= accuracy_score(np.array(target_multi_hot), np.array(predicted_label))
            predict_label_tensor = torch.tensor(predicted_label)
            predict_label_tensor = predict_label_tensor.cuda()
            acc = accuracy(predict_label_tensor,target_multi_hot)

            # ave_precision_score = average_precision_score(y_true=np.array(target_multi_hot_numpy),
                                              # y_score=np.array(sigmoid_op), average='micro')
            
            return [acc]

   

In [ ]:


# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny', do_lower_case=True)

processor = DocumentDataPreprocessor(
    tokenizer,
    formatter
)


Loading BERT tokenizer...


In [ ]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [ ]:
import pandas
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
def create_tensor_dataset(processor,json_arr):
  fmt = DataIntegrationFormatter()
  formatted_content = map(lambda x: fmt(x['value']),json_arr)
  input_ids,att_mask = processor.prepare_text_tensors(formatted_content,max_length=40) 
  label_list = []
  num_labels = []
  for d in json_arr:
    label_list.append([0 if i not in d['label_index'] else 1 for i in range(1,NUM_LABELS+1)])
    num_labels.append(len(d['label_index']))

  label_tensor = torch.Tensor(label_list).type(torch.FloatTensor)
  num_label_tensor = torch.Tensor(num_labels)
  return TensorDataset(input_ids,att_mask,label_tensor,num_label_tensor)

train_dataset = create_tensor_dataset(processor,train_data_json)
# test_tensor_set = create_tensor_dataset(processor,test_data_json)

In [ ]:
import random
class DataIntegrationClassificationDataModule(pl.LightningDataModule):
    # text : should 
    def __init__(self,tensor_dataset,batch_size:int,):
        super().__init__()
        self.batch_size = batch_size
        self.tensor_dataset = tensor_dataset
        # self.records = random.sample(text,30) # TODO change this. 
    def get_full_tensorset(self):
        return self.tensor_dataset

    def setup(self, stage=None):
        train_set,val_set = DocumentDataPreprocessor.split_dataset(self.tensor_dataset,train_percent=0.8)
        self.train_set = train_set
        self.val_set  =val_set 

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size,shuffle=True)

In [ ]:
model = GPT2DataIntegrationClassificationModel('prajjwal1/bert-tiny',tokenizer)
for param in model.classification_transformer.classification_head.transformer.parameters():
    param.requires_grad = False


Got ff_layers 1280 11475


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BERTClassifier: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BERTClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BERTClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BERTClassifier were not initialized from the model checkpoint at prajjwal1/bert-tiny and a

In [ ]:
data_module = DataIntegrationClassificationDataModule(train_dataset,128)


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger,TensorBoardLogger
print(f"Using {torch.cuda.device_count()} GPUs")
epochs = MAX_EPOCHS
model_checkpoint = ModelCheckpoint(
    save_last=False,
    mode='min',
    prefix='data-integration-gpt-2-',
    verbose=True,
)
trainer = pl.Trainer(
                     max_epochs=epochs,\
                     progress_bar_refresh_rate=5,\
                     gpus=1
                     
                     )
                    #  checkpoint_callback = model_checkpoint,\
%time trainer.fit(model, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Using 1 GPUs


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name                       | Type                                        | Params
-------------------------------------------------------------------------------------------
0 | classification_transformer | DataIntegrationClassificationMultiHeadModel | 15 M  
1 | loss_fn                    | BCEWithLogitsLoss                           | 0     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, val dataloader 0, does not have 

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Saving latest checkpoint..



CPU times: user 4min 47s, sys: 7.33 s, total: 4min 54s
Wall time: 4min 59s


1

In [ ]:
from torch.utils.data import DataLoader
test_tensor_set = create_tensor_dataset(processor,test_data_json)
%time trainer.test(model, test_dataloaders=DataLoader(test_tensor_set,batch_size=128))

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'epoch_test_acc': tensor(0.9997),
 'epoch_test_loss': tensor(0.6898, device='cuda:0')}
--------------------------------------------------------------------------------

CPU times: user 29.8 s, sys: 617 ms, total: 30.5 s
Wall time: 30.4 s


[{'epoch_test_acc': 0.9997385740280151, 'epoch_test_loss': 0.6897561550140381}]